# Overview Thesis Code

This notebook provides a comprehensive overview of the code used in my master's thesis.

Access to the thesis code is available via the GitHub repository linked here: [\url{https/github.com/jacobineanna/masterthesis}]. Initially, the code was stored in separate Python files. For clarity and ease of understanding, it has been consolidated into a single Jupyter notebook.

The core structure of the code draws inspiration from formal Data Science curriculum courses. Additionally, some online sources were used. Any reused or adapted code segments are clearly identified within this notebook. These adaptations stem from various sources, including a Medium article on raincloud plots (Belengeanu, 2022) and an online article on Towards Data Science focusing on SHAP analysis for binary classification (O'Sullivan, 2023). Additionally, ChatGPT has been employed as a debugging tool to rectify coding errors (OpenAI, 2022).

## Importing all neccessary packages

In [ ]:
"""
IMPORTING ALL NECCESSARY PACKAGES/LIBRARIES
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from geopy.distance import geodesic
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, make_scorer, confusion_matrix
from skopt import BayesSearchCV
import seaborn as sns

import json
import pickle
import shap


### Preprocessing

Consists of:
- Deletion of variables too close to target
- Deletion of irrelevant variables
- Dealing with missing values
- Dataset enrichment (creation of train station related variables)
- Creating binary target variable

In [ ]:
"""
DELETING VARIABLES
"""
df = pd.read_csv('Datasets/ODiN2022_Databestand.csv',
                 delimiter=';', encoding='latin1', na_values='#NULL!')

# List of columns dropped
columns_to_drop = [
    'OP',
    'OPID',
    'Steekproef',
    'Mode',
    'Corop',
    'BuurtAdam',
    'MRA',
    'MRDH',
    'Utr',
    'BouwjaarPa1',
    'KBouwjaarPa1',
    'KGewichtPa1',
    'TenaamPa1',
    'BrandstofPa2',
    'XBrandstofPa2',
    'BrandstofEPa2',
    'BouwjaarPa2',
    'KBouwjaarPa2',
    'KGewichtPa2',
    'TenaamPa2',
    'BouwjaarPaL',
    'KBouwjaarPaL',
    'KGewichtPaL',
    'Jaar',
    'EFiets',
    'AutoLPl',
    'AutoBed',
    'AutoDOrg',
    'AutoDPart',
    'AutoDBek',
    'AutoLeen',
    'AutoHuur',
    'AutoAnd',
    'ByzDag',
    'ByzAdr',
    'ByzVvm',
    'ByzTyd',
    'ByzDuur',
    'ByzRoute',
    'ByzReden',
    'Verpl',
    'VerplID',
    'VerplNr',
    'MeerWink',
    'VertGeb',
    'VertPCBL',
    'VertCorop',
    'VertMRA',
    'VertMRDH',
    'VertUtr',
    'AankGeb',
    'AankPCBL',
    'AankCorop',
    'AankMRA',
    'AankMRDH',
    'AankUtr',
    'PCG',
    'GemG',
    'PCBLG',
    'HvmRol',
    'KHvm',
    'VolgWerk',
    'SAantAdr',
    'SDezPlts',
    'SPlaats1',
    'SPlaats2',
    'SPlaats3',
    'SPlaats4',
    'SPlaats5',
    'AfstS',
    'AfstSBL',
    'SVvm1',
    'SVvm2',
    'SVvm3',
    'SVvm4',
    'SBegUur',
    'SBegMin',
    'SEindUur',
    'SEindMin',
    'CorrVerpl',
    'GehBLVer',
    'Rit',
    'RitID',
    'RitNr',
    'AfstRBL',
    'Rvm',
    'KRvm',
    'RReisduurBL',
    'RCorrSnelh',
    'RVliegVer',
    'FactorH',
    'FactorP',
    'FactorV'
    'Rvm',
    'AutoEig',
    'AutoHhl',
    'AutoLWg',
    'WrkVervw',
    'RvmRol',
    'RAantIn',
    'KRvm']

# Dropping irrelevant variables/variables that represent target variable directly
df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
"""
DEALING WITH MISSING VALUES
"""
df.dropna(inplace=True)

print("Number of missing values for each variable:")
for column in df.columns:
    print(f"{column}: {df[column].isnull().sum()}")

# Printing the number of rows and columns remained
print("Number of rows remained:", df.shape[0])
print("Number of columns remainded:", df.shape[1])

# Saving DataFrame to a CSV file
df.to_csv('Datasets/participants_dataset.csv', index=False)

In [ ]:
"""
CREATING VARIABLES TRAIN STATION PROXIMITY, NEAREST TRAINSTATION AND TYPE OF TRAINSTATION

Final variables:
- nearest_station_encoded: the nearest trainstation to entries homeadress label encoded
- station_type_encoded: type of nearest station
- distance_to nearest_station_km: distance to nearest station in km (later normalized)
"""

# Loading the datasets used for creation of new variables
participants_df = pd.read_csv('Datasets/participants_dataset.csv')              # Dataset 1: Participants with postal codes
trainstations_df = pd.read_csv('Datasets/trainstations_dataset.csv')            # Dataset 2: Train stations with coordinates
postalcodes_df = pd.read_csv('Datasets/postalcodes_dataset.csv', delimiter=';') # Dataset 3: Postal codes with coordinates

# Splitting the first column into latitude and longitude and dropping the original column
postalcodes_df[['Latitude', 'Longitude']] = postalcodes_df.iloc[:, 0].str.split(',', expand=True)
postalcodes_df.drop(postalcodes_df.columns[0], axis=1, inplace=True)

# Merging postalcodes_df and participants_df to get participant coordinates
participants_with_coordinates = pd.merge(participants_df, postalcodes_df[['PC4', 'Latitude', 'Longitude']], left_on='WoPC', right_on='PC4', how='left')
participants_with_coordinates.drop(columns=['PC4'], inplace=True)

# Initialising lists to store the nearest train station, the distance and the type of station for each participant
nearest_station = []            # Initialising a list to store nearest station
distance_to_station = []        # Initialising a list to store distance to nearest station
station_type = []               # Initialising a list to store type of nearest station

for index, row in participants_with_coordinates.iterrows():     # Iterating over each participant
    participant_coords = (row['Latitude'], row['Longitude'])
    min_distance = float('inf')
    nearest = None
    station_type_nearest = None

    for _, station_row in trainstations_df.iterrows():          # Iterating over each train station
        station_coords = (station_row['geo_lat'], station_row['geo_lng'])
        distance = geodesic(participant_coords, station_coords).kilometers

        # If current station is closer than the previous closest station, updating min_distance, nearest and station_type_nearest
        if distance < min_distance:
            min_distance = distance
            nearest = station_row['name_long']
            station_type_nearest = station_row['type']

    # Appending the results of each participant to the lists
    nearest_station.append(nearest)
    distance_to_station.append(min_distance)
    station_type.append(station_type_nearest)

    # Progress massage for every 1000 participants
    if (index + 1) % 1000 == 0:
        print(f"Processed {index + 1} participants...")

# Adding the resulting lists to the participants_with_coordinates df
participants_with_coordinates['nearest_station'] = nearest_station
participants_with_coordinates['distance_to_station_km'] = distance_to_station
participants_with_coordinates['station_type'] = station_type

# Label Encoding and dropping original columns
label_encoder = LabelEncoder()
participants_with_coordinates['nearest_station_encoded'] = label_encoder.fit_transform(participants_with_coordinates['nearest_station'])
participants_with_coordinates['station_type_encoded'] = label_encoder.fit_transform(participants_with_coordinates['station_type'])

participants_with_coordinates.drop(columns=['nearest_station'], inplace=True)
participants_with_coordinates.drop(columns=['station_type'], inplace=True)

# Saving the dataframe to csv
participants_with_coordinates.to_csv('Datasets/preprocessed_data.csv', index=False)

# Progress message
print("Preprocessed data saved successfully.")

In [ ]:
"""
NORMALISING NUMERICAL VARIABLES
"""

df_to_normalise = pd.read_csv("preprocessed_data.csv")
print(df_to_normalise.head(10))

# Initialising the MinMaxScaler
scaler = MinMaxScaler()

# Specifying the numerical variables that need to be normalised
variables_to_normalise = ['AantVpl',
                          'AantOVVpl',
                          'AantSVpl',
                          'ReisduurOP',
                          'AfstandOP',
                          'AfstandSOP',
                          'AfstV',
                          'AfstR',
                          'VertUur',
                          'VertMin',
                          'AankUur',
                          'AankMin',
                          'Reisduur',
                          'RVertUur',
                          'RVertMin',
                          'RAankUur',
                          'RAankMin',
                          'RReisduur',
                          'Latitude',
                          'Longitude',
                          'distance_to_station_km']

# Fitting and transforming the selected variables
df_to_normalise[variables_to_normalise] = scaler.fit_transform(df_to_normalise[variables_to_normalise])

# Displaying the normalised dataframe
print(df_to_normalise.head(10))

# Saving normalised dataframe
df_to_normalise.to_csv('Datasets/normalised_dataset.csv', index=False)

In [ ]:
"""
CREATING BINARY TARGET VARIABLE
"""

df = pd.read_csv('Datasets/normalised_dataset.csv')

# Specifying the labels for sustainable and not sustainable transportation modes
sustainable_labels = {
    2: 'Trein',
    3: 'Bus',
    4: 'Tram',
    5: 'Metro',
    6: 'Speedpedelec',
    7: 'Elektrische fiets',
    8: 'Niet-elektrische fiets',
    9: 'Te voet',
    19: 'Gehandicaptenvervoermiddel met motor',
    20: 'Gehandicaptenvervoermiddel zonder motor',
    21: 'Skates/skeelers/step',
    24: 'Anders zonder motor'
}

not_sustainable_labels = {
    1: 'Personenauto',
    10: 'Touringcar',
    11: 'Bestelauto',
    12: 'Vrachtwagen',
    13: 'Camper',
    14: 'Taxi/Taxibusje',
    15: 'Landbouwvoertuig',
    16: 'Motor',
    17: 'Bromfiets',
    18: 'Snorfiets',
    22: 'Boot',
    23: 'Anders met motor',
}
# Categorise the transportation modes into binary target variable
df['category'] = df['Hvm'].map(lambda x: 'Sustainable' if x in sustainable_labels else 'Not Sustainable')

# Encoding the categories
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])
df.drop(columns=['category'], inplace=True)

print(df.head(5))
print(df.shape)

# Saving final preprocessed dataset
df.to_csv('Datasets/final_preprocessed_data.csv', index=False)

### Exploratory Data Analysis

Consists of:
- Pie chart HVM
- Pie charts per Gender
- Pie chart binary target variable
- Raincloud plot age/target variable

In [ ]:
"""
Pie chart hoofdvervoermiddel
"""
df = pd.read_csv('Datasets/final_preprocessed_data.csv')

# Converting 'Hvm' column to numeric and counting occurences
df['Hvm'] = pd.to_numeric(df['Hvm'], errors='coerce')
vervoer_soorten = df['Hvm'].value_counts()

# Selecting the top 5 and combining the rest into a category called 'Other' (for the sake of readability)
top_5 = vervoer_soorten.head(5)
other_transportation_modes = vervoer_soorten[~vervoer_soorten.index.isin(top_5.index)].sum()
vervoer_soorten_combined = pd.concat([top_5, pd.Series({'Other': other_transportation_modes.sum()})])

# Specifying the labels
labels = {
    1: 'Personenauto',
    2: 'Trein',
    3: 'Bus',
    4: 'Tram',
    5: 'Metro',
    6: 'Speedpedelec',
    7: 'Elektrische fiets',
    8: 'Niet-elektrische fiets',
    9: 'Te voet',
    10: 'Touringcar',
    11: 'Bestelauto',
    12: 'Vrachtwagen',
    13: 'Camper',
    14: 'Taxi/Taxibusje',
    15: 'Landbouwvoertuig',
    16: 'Motor',
    17: 'Bromfiets',
    18: 'Snorfiets',
    19: 'Gehandicaptenvervoermiddel met motor',
    20: 'Gehandicaptenvervoermiddel zonder motor',
    21: 'Skates/skeelers/step',
    22: 'Boot',
    23: 'Anders met motor',
    24: 'Anders zonder motor'
}
labels_for_pie = vervoer_soorten_combined.index.map(lambda x: labels[x] if x in labels else 'Other')

# Setting colours for the pie chart
colours = ['#FDE725FF', '#95D840FF', '#55C667FF', '#29AF7FFF', '#238A8DFF', '#39568CFF']

# Plot the pie chart with custom labels
plt.pie(vervoer_soorten_combined, labels=labels_for_pie, colors=colours, autopct='%1.1f%%', startangle=90)
plt.title('Distribution of Main Mode of Transport')
plt.xticks(rotation='vertical')
plt.rcParams['font.size'] = 14
plt.tight_layout()
plt.savefig('Plots/distribution_main_mode_of_transport.pdf')

In [ ]:
"""
Pie charts per gender
"""
df = pd.read_csv('Datasets/final_preprocessed_data.csv')

# Filtering data for males and females
males_data = df[df['Geslacht'] == 1]
females_data = df[df['Geslacht'] == 2]

# Calculating sustainable and not sustainable transportation percentages
males_counts = males_data['category_encoded'].value_counts(normalize=True) * 100
females_counts = females_data['category_encoded'].value_counts(normalize=True) * 100

# Creating a figure with two subplots (pie charts per gender)
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Pie chart males
axes[0].pie(males_counts, labels=['Not Sustainable', 'Sustainable'],
            autopct='%1.1f%%', colors=['#FDE725FF', '#20A387FF'], startangle=90)
axes[0].set_title('Males')

# Pie chart females
axes[1].pie(females_counts, labels=['Not Sustainable', 'Sustainable'],
            autopct='%1.1f%%', colors=['#FDE725FF', '#20A387FF'], startangle=90)
axes[1].set_title('Females')

fig.suptitle('Transportation Distribution by Gender', fontsize=16)
plt.savefig('Plots/transportation_distribution_by_gender.pdf')

In [ ]:
"""
Pie Chart binary target variable
"""
df = pd.read_csv('Datasets/final_preprocessed_data.csv')

# Converting 'Hvm' column to numeric and counting occurences
df['category_encoded'] = pd.to_numeric(df['category_encoded'], errors='coerce')
vervoer_soorten = df['category_encoded'].value_counts()

# Specifying labels
labels = {
    0: 'Not Sustainable \n Transportation',
    1: 'Sustainable \n Transportation',
}

labels_for_pie = vervoer_soorten.index.map(lambda x: labels.get(x, 'Unknown'))

# Setting properties
colors = ['#20A387FF', '#FDE725FF']
plt.rcParams.update({'font.family': 'Times New Roman', 'font.size': 16})

# Pie chart binary target variable
plt.pie(vervoer_soorten, labels=labels_for_pie, colors=colors, autopct='%1.1f%%', startangle=90)
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.savefig('Plots/distribution_sustainable_not_sustainable.pdf')

In [ ]:
"""
Custom Plot to create raincloud plot

    Input:
        data: list of 1D numpy arrays containing feature values
        names: list of length 2 with feature names
        (optional) max_scatters: maximum number of scatter points at each unique feature
    Output:
        fig: figure object, use e.g. plt.show() and plt.savefig after retrieving this function output
    
    Adapted from: https://medium.com/@alexbelengeanu/getting-started-with-raincloud-plots-in-python-2ea5c2d01c11
"""

def make_it_rain(data,names,max_scatters=10):

    # Setting font properties for the entire plot
    plt.rcParams.update({'font.family': 'Times New Roman', 'font.size': 16})
    
    # Creating the figure
    fig, ax = plt.subplots(figsize=(10,4))

    # Boxplot
    bp = ax.boxplot(data, patch_artist = True, vert = False)

    boxplots_colors = ['#FDE725FF', '#FDE725FF']
    for patch, color in zip(bp['boxes'], boxplots_colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.4)

    # Violinplot
    vp = ax.violinplot(data, points=500, 
                showmeans=False, showextrema=False, showmedians=False, vert=False)
    
    violin_colors = ['#29AF7FFF', '#29AF7FFF']
    for idx, b in enumerate(vp['bodies']):
        # Get the center of the plot
        m = np.mean(b.get_paths()[0].vertices[:, 0])
        # Modifying to only see the upper half of the violin plot
        b.get_paths()[0].vertices[:, 1] = np.clip(b.get_paths()[0].vertices[:, 1], idx+1, idx+2)
        b.set_color(violin_colors[idx])

    # Scatter plot
    scatter_colors = ['#39568CFF', '#39568CFF']

    for idx, feature in enumerate(data):
        # Only take a portion of the data to make the data not too dense
        feature_uniques, feature_counts = np.unique(feature, return_counts=True)
        feature_counts_dict = dict(zip(feature_uniques, feature_counts))
        feature_counts_capped = np.empty((0,0), int)
        for feature_unique_current in feature_uniques:
            feature_count_current = feature_counts_dict[feature_unique_current]
            if feature_count_current > max_scatters:
                feature_counts_capped = np.append(feature_counts_capped,np.full(max_scatters,feature_unique_current))
            else:
                feature_counts_capped = np.append(feature_counts_capped,np.full(feature_count_current,feature_unique_current))
        
        # Creating initial scatter positions without randomisation
        scatter_height = np.full(len(feature_counts_capped), idx + .8).astype(float)
        # Adding random jitter effect so the features do not overlap on the y-axis
        feature_capped_idxs = np.arange(len(scatter_height))
        scatter_height.flat[feature_capped_idxs] += np.random.uniform(low=-.05, high=.05, size=len(feature_capped_idxs))
        # Draw scatters
        plt.scatter(feature_counts_capped, scatter_height, s=.3, c=scatter_colors[idx])

    plt.yticks(np.arange(1,3,1), names, fontsize=16)  # Set text labels
    plt.tight_layout()
    
    return fig

In [ ]:
"""
Raincloud plot Age and Target Variable
"""
df = pd.read_csv('Datasets/final_preprocessed_data.csv')

# Filtering the data per category and than combining into one variable
age_sustainable = df[df['category_encoded'] == 1]['Leeftijd'].to_numpy()
age_not_sustainable = df[df['category_encoded'] == 0]['Leeftijd'].to_numpy()
age_data = [age_not_sustainable,age_sustainable]

# Set figure metadata
feature_names = ["Not sustainable","Sustainable"]
max_scatters = 10

# Make it rain
fig = cp.make_it_rain(age_data,feature_names,max_scatters)

plt.savefig('Plots/raincloud_age_category.pdf')
plt.show()

### Data Splitting

In [ ]:
data = pd.read_csv('Datasets/final_preprocessed_data.csv')

# Drop 'Hvm'column
data.drop('Hvm', axis=1, inplace=True)
print("Data shape after dropping 'Hvm':", data.shape)

# Splitting the data in X and y
y = data['category_encoded']
X = data.drop('category_encoded', axis=1)

# Splittingthe data into training/validation and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Saving the data splits into separate CSV files
X_train.to_csv('Datasets/X_train.csv', index=False)
y_train.to_csv('Datasets/y_train.csv', index=False)
X_test.to_csv('Datasets/X_test.csv', index=False)
y_test.to_csv('Datasets/y_test.csv', index=False)

print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

### Training the initial models

- Default Decision Tree
- Default Random Forest
- Deafault Gradient Boosting
- Default ANN

In [ ]:
"""
Default Decision Tree
"""
X_trainval = pd.read_csv('Datasets/X_train.csv')
y_trainval = pd.read_csv('Datasets/y_train.csv')

# Extra split to create validation set
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, stratify=y_trainval, random_state=42)

# Decision Tree Classifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_val)

# Calculating evaluation metrics
balanced_accuracy = balanced_accuracy_score(y_val, y_pred)
print(f"Balanced Accuracy Default Decision Tree: {balanced_accuracy}")

precision = precision_score(y_val, y_pred, average='macro')
print(f"Precision: {precision}")

recall = recall_score(y_val, y_pred, average='macro')
print(f"Recall: {recall}")

In [ ]:
"""
Default Random Forest
"""

X_trainval = pd.read_csv('Datasets/X_train.csv')
y_trainval = pd.read_csv('Datasets/y_train.csv')

# Extra split to create validation set
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, stratify=y_trainval, random_state=42)

# Random Forest Classifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_val)

# Calculating evaluation metrics
balanced_accuracy = balanced_accuracy_score(y_val, y_pred)
print(f"Balanced Accuracy Default Random Forest: {balanced_accuracy}")

precision = precision_score(y_val, y_pred, average='macro')
print(f"Precision Default Random Forest: {precision}")

recall = recall_score(y_val, y_pred, average='macro')
print(f"Recall Default Random Forest: {recall}")

In [ ]:
"""
Default Gradient Boosting
"""

X_trainval = pd.read_csv('Datasets/X_train.csv')
y_trainval = pd.read_csv('Datasets/y_train.csv')

# Extra split to create validation set
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, stratify=y_trainval, random_state=42)

# Gradient Boosting Classifier
gradient_boosting = GradientBoostingClassifier(n_estimators=100, random_state=42)
gradient_boosting.fit(X_train, y_train.values.ravel())
y_pred = gradient_boosting.predict(X_val)

# Calculating evaluation metrics
balanced_accuracy = balanced_accuracy_score(y_val, y_pred)
print(f"Balanced Accuracy Default Gradient Boosting Classifier: {balanced_accuracy}")

precision = precision_score(y_val, y_pred, average='macro')
print(f"Precision Deafault Gradient Boosting Classifier: {precision}")

recall = recall_score(y_val, y_pred, average='macro')
print(f"Recall Default Gradient Boosting Classifier: {recall}")

In [ ]:
"""
Default ANN
"""

X_trainval = pd.read_csv('Datasets/X_train.csv')
y_trainval = pd.read_csv('Datasets/y_train.csv')

# Extra split to create validation set
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, stratify=y_trainval, random_state=42)

# Convert y_train to numpy array and flatten for binary classification
y_train = np.array(y_train).ravel()

# Function creating default ANN model
def create_ann_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.fit_transform(X_val)

# ANN Classifier
model = create_ann_model()
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=0)
y_pred_proba = model.predict(X_val_scaled)
y_pred = (y_pred_proba > 0.5).astype(int).ravel()  # Prob to binary (0/1)

# Calculating evaluation metrics
balanced_accuracy = balanced_accuracy_score(y_val, y_pred)
print(f"Balanced Accuracy Default ANN: {balanced_accuracy}")

precision = precision_score(y_val, y_pred, average='macro')
print(f"Precision Deafault ANN: {precision}")

recall = recall_score(y_val, y_pred, average='macro')
print(f"Recall Default ANN: {recall}")

### Hyperparameter Tuning

In [ ]:
"""
Optimisation of Decision Tree Classifier
"""

X_train = pd.read_csv('Datasets/X_train.csv')
y_train = pd.read_csv('Datasets/y_train.csv')

# Search Space Hyperparameters
search_space_hyperparameters = {
    'max_depth': (1, 150),              # Maximum depth of the tree
    'min_samples_split': (2, 50),       # Minimum number of samples required to split an internal node
    'min_samples_leaf': (1, 100),       # Minimum number of samples required to be at a leaf node
}


# Bayesian Optimisation of Decision Tree Classifier
decision_tree = DecisionTreeClassifier()
scorer = make_scorer(balanced_accuracy_score)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

bayesian_opt = BayesSearchCV(
    estimator=decision_tree,
    search_spaces=search_space_hyperparameters,
    n_iter=50,                        # Number of parameter settings that are sampled
    cv=skf,                            # Number of cross-validation folds
    n_jobs=-1,                         # Use all available CPU cores
    scoring=scorer,                    # Metric to optimize
    random_state=42,                   # Random seed for reproducibility
    verbose=1,
)

bayesian_opt.fit(X_train, y_train)

# Printing the best hyperparameters and the corresponding metrics
print("Best hyperparameters Decision Tree:")
print(bayesian_opt.best_params_)
print("Best balanced accuracy Decision Tree:", bayesian_opt.best_score_)
best_hyperparameters = bayesian_opt.best_params_

# Storing the best hyperparameters
with open('best_hyperparameters_decision_tree.json', 'w') as f:
    json.dump(best_hyperparameters, f)

In [ ]:
"""
Confusion Matrix and Evaluation Metrics of Final Decision Tree
"""

X_train = pd.read_csv('Datasets/X_train.csv')
y_train = pd.read_csv('Datasets/y_train.csv')
X_test = pd.read_csv('Datasets/X_test.csv')
y_test = pd.read_csv('Datasets/y_test.csv')

with open('best_hyperparameters_decision_tree.json', 'r') as f:
    loaded_best_hyperparameters = json.load(f)

# Final Decision Tree
best_decision_tree = DecisionTreeClassifier(**loaded_best_hyperparameters, random_state=42)
best_decision_tree.fit(X_train, y_train.values.ravel())
y_pred = best_decision_tree.predict(X_test)

# Evaluation metrics
balanced_accuracy_loaded = balanced_accuracy_score(y_test, y_pred)
recall_loaded = recall_score(y_test, y_pred, average='weighted')
precision_loaded = precision_score(y_test, y_pred, average='weighted')

print("Performance on the test set with loaded best hyperparameters:")
print("Balanced Accuracy:", balanced_accuracy_loaded)
print("Recall:", recall_loaded)
print("Precision:", precision_loaded)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Setting font properties
plt.rcParams.update({'font.family': 'Times New Roman', 'font.size': 16})

# Plot confusion matrix
plt.figure()
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig('Plots/confusion_matrix_decision_tree.pdf')
plt.show()

In [ ]:
"""
Optimisation of Random Forest Classifier
"""

X_train = pd.read_csv('Datasets/X_train.csv')
y_train = pd.read_csv('Datasets/y_train.csv')


# Search Space Hyperparameters
search_space_hyperparameters = {
    'n_estimators': (2, 500),
    'max_depth': (1, 150),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20)
}

# Bayesian Optimisation of Random Forest Classifier
random_forest = RandomForestClassifier(random_state=42)
scorer = make_scorer(balanced_accuracy_score)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

bayesian_search = BayesSearchCV(
    estimator=random_forest,
    search_spaces=search_space_hyperparameters,
    n_iter=50,                         # Number of parameter settings that are sampled
    cv=skf,                            # Number of cross-validation folds
    n_jobs=-1,                         # Use all available CPU cores
    scoring=scorer,                    # Metric to optimize
    random_state=42,                   # Random seed for reproducibility
    verbose=1
)

bayesian_search.fit(X_train, y_train.values.ravel())

# Printing the best hyperparameters and the corresponding metrics
print("Best hyperparameters for Random Forest found using Bayesian optimization:")
print(bayesian_search.best_params_)
print("Best cross-validation score Random Forest using Bayesian optimization:", bayesian_search.best_score_)
best_hyperparameters = bayesian_search.best_params_

# Storing the best hyperparameters
with open('best_hyperparameters_random_forest.json', 'w') as f:
    json.dump(best_hyperparameters, f)

In [ ]:
"""
Confusion Matrix and Evaluation Metrics of Final Random Forest
"""

X_train = pd.read_csv('Datasets/X_train.csv')
y_train = pd.read_csv('Datasets/y_train.csv')
X_test = pd.read_csv('Datasets/X_test.csv')
y_test = pd.read_csv('Datasets/y_test.csv')

with open('best_hyperparameters_random_forest.json', 'r') as f:
    best_hyperparameters = json.load(f)

# Final Random Forest
best_random_forest = RandomForestClassifier(**best_hyperparameters, random_state=42)
best_random_forest.fit(X_train, y_train.values.ravel())
y_pred = best_random_forest.predict(X_test)

# Evaluation metrics
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')

print("Performance on the test set:")
print("Balanced Accuracy Random Forest:", balanced_accuracy)
print("Recall Random Forest:", recall)
print("Precision Random Forest:", precision)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Setting Font Properties
plt.rcParams.update({'font.family': 'Times New Roman', 'font.size': 16})

# Plot Confusion Matrix
plt.figure()
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig('Plots/confusion_matrix_random_forest.pdf')
plt.show()

In [ ]:
"""
Optimisation of Gradient Boosting Classifier
"""

X_train = pd.read_csv('Datasets/X_train.csv')
y_train = pd.read_csv('Datasets/y_train.csv')

# Search Space Hyperparameters
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [7, 9, 11],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 10, 20]
}

# Grid Search for gradient boosting
gradient_boosting = GradientBoostingClassifier(random_state=42)
scorer = make_scorer(balanced_accuracy_score)
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)


grid_search = GridSearchCV(estimator=gradient_boosting, param_grid=param_grid, cv=skf, scoring=scorer, verbose=2)
grid_search.fit(X_train, y_train.values.ravel())  # ravel y_train to avoid DataConversionWarning

# Evaluation Metrics
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

print("Best Hyperparameters:", best_params)
print("Best Balanced Accuracy:", best_accuracy)

# Storing the best hyperparameters
with open('best_hyperparameters_gradient_boosting.json', 'w') as f:
    json.dump(best_params, f)

In [ ]:
"""
Confusion Matrix and Evaluation Metrics of Final Gradient Boosting
"""

X_train = pd.read_csv('Datasets/X_train.csv')
y_train = pd.read_csv('Datasets/y_train.csv')
X_test = pd.read_csv('Datasets/X_test.csv')
y_test = pd.read_csv('Datasets/y_test.csv')

with open('best_hyperparameters_gradient_boosting.json', 'r') as f:
    loaded_best_hyperparameters = json.load(f)

# Final Gradient Boosting
best_random_forest = GradientBoostingClassifier(**loaded_best_hyperparameters, random_state=42)
best_random_forest.fit(X_train, y_train.values.ravel())
y_pred = best_random_forest.predict(X_test)

# Evaluation metrics
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')

print("Performance on the test set:")
print("Balanced Accuracy:", balanced_accuracy)
print("Recall:", recall)
print("Precision:", precision)

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Set font properties
plt.rcParams.update({'font.family': 'Times New Roman', 'font.size': 16})

# Plot confusion matrix
plt.figure()
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig('Plots/confusion_matrix_gradient_boosting.pdf')
plt.show()

In [ ]:
"""
Optimalisation of ANN

Automatically tuned on param grid, and manually tuned number of epochs and batch size

"""

X_train = pd.read_csv('Datasets/X_train.csv')
y_train = pd.read_csv('Datasets/y_train.csv')

# Converting y_train to numpy array and flatten for binary classification
y_train = np.array(y_train).ravel()

# Initialise Stratified K-Fold with k=10
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Function to create ANN model with dropout and variable number of layers
def create_ann_model(hidden_layer_size=64, learning_rate=0.001, dropout_rate=0.0, num_layers=2,
                     activation_function='relu', regularization=None, optimizer='adam'):
    model = Sequential()
    model.add(Dense(hidden_layer_size, activation=activation_function, input_shape=(X_train.shape[1],),
                    kernel_regularizer=regularization))
    model.add(Dropout(dropout_rate))
    
    for _ in range(num_layers - 1):  # Add additional hidden layers
        model.add(Dense(hidden_layer_size, activation=activation_function, kernel_regularizer=regularization))
        model.add(Dropout(dropout_rate))
    
    model.add(Dense(1, activation='sigmoid'))
    
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Search Space Hyperparameters
param_grid = {
    'hidden_layer_size': [32],
    'learning_rate': [0.01],
    'dropout_rate': [0.1],
    'num_layers': [2, 4],
    'activation_function': ['relu', 'sigmoid'],
    'regularization': [None, 'l1'],
    'optimizer': ['adam']
}

best_accuracy = 0
best_params = None

# Grid search ANN
for params in ParameterGrid(param_grid):
    fold_accuracy = []
    print("Current parameters:", params)
    
    for train_index, val_index in skf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
        
        # Standardize the input features
        scaler = StandardScaler()
        X_train_fold_scaled = scaler.fit_transform(X_train_fold)
        X_val_fold_scaled = scaler.transform(X_val_fold)
        
        # Training the ANN model
        model = create_ann_model(hidden_layer_size=params['hidden_layer_size'],
                                 learning_rate=params['learning_rate'],
                                 dropout_rate=params['dropout_rate'],
                                 num_layers=params['num_layers'],
                                 activation_function=params['activation_function'],
                                 regularization=params['regularization'],
                                 optimizer=params['optimizer'])
        model.fit(X_train_fold_scaled, y_train_fold, epochs=10, batch_size=32, verbose=0)
        
        # Evaluating the model on the validation fold
        y_pred_proba = model.predict(X_val_fold_scaled)
        y_pred = (y_pred_proba > 0.5).astype(int).ravel()  # Prob to binary (0/1)
        balanced_accuracy = balanced_accuracy_score(y_val_fold, y_pred)
        fold_accuracy.append(balanced_accuracy)
    
    # Calculating the mean accuracy over all folds
    mean_accuracy = np.mean(fold_accuracy)
    print(f"Mean Accuracy for current parameters: {mean_accuracy}")
    
    # Updating best parameters if current parameters result in better accuracy
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_params = params

# Evaluation Metrics
print("Best parameters:", best_params)
print("Best accuracy:", best_accuracy)

# Store the best hyperparameters
with open('best_hyperparameters_ann.json', 'w') as f:
    json.dump(best_params, f)

In [1]:
"""
Confusion Matrix and Evaluation Metrics of Final ANN
"""

# Random seed for reproducibility
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

X_train = pd.read_csv('Datasets/X_train.csv')
y_train = pd.read_csv('Datasets/y_train.csv')
X_test = pd.read_csv('Datasets/X_test.csv')
y_test = pd.read_csv('Datasets/y_test.csv')


# Convert y_train to numpy array and flatten for binary classification
y_train = np.array(y_train).ravel()

# Function to manually tune ANN to best hyperparametersettings
def create_ann_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

model = create_ann_model()
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=0)
y_pred_proba = model.predict(X_test_scaled)
y_pred = (y_pred_proba > 0.5).astype(int).ravel()  # Prob to binary (0/1)

# Evaluation Metrics
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy Default ANN: {balanced_accuracy}")
precision = precision_score(y_test, y_pred, average='macro')
print(f"Precision Default ANN: {precision}")
recall = recall_score(y_test, y_pred, average='macro')
print(f"Recall Default ANN: {recall}")

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Set font properties
plt.rcParams.update({'font.family': 'Times New Roman', 'font.size': 16})

# Plot confusion matrix
plt.figure()
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig('Plots/confusion_matrix_ann.pdf')
plt.show()

''

### Error Pattern Analysis

In [ ]:
X_train = pd.read_csv('Datasets/X_train.csv')
y_train = pd.read_csv('Datasets/y_train.csv')
X_test = pd.read_csv('Datasets/X_test.csv')
y_test = pd.read_csv('Datasets/y_test.csv')

with open('best_hyperparameters_gradient_boosting.json', 'r') as f:
    loaded_best_hyperparameters = json.load(f)

# Train and fit tuned gradient boosting classifier
best_gradient_boosting = GradientBoostingClassifier(**loaded_best_hyperparameters, random_state=42)
best_gradient_boosting.fit(X_train, y_train.values.ravel())
y_pred = best_gradient_boosting.predict(X_test)

# Calculate overall evaluation metrics
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')

# Separating the data by gender
male_indices = X_test[X_test['Geslacht'] == 1].index
female_indices = X_test[X_test['Geslacht'] == 2].index

X_test_male = X_test.loc[male_indices].drop(columns=['Geslacht'])
y_test_male = y_test.loc[male_indices]
y_pred_male = y_pred[male_indices]

X_test_female = X_test.loc[female_indices].drop(columns=['Geslacht'])
y_test_female = y_test.loc[female_indices]
y_pred_female = y_pred[female_indices]

# Evaluation metrics for males
balanced_accuracy_male = balanced_accuracy_score(y_test_male, y_pred_male)
recall_male = recall_score(y_test_male, y_pred_male, average='weighted')
precision_male = precision_score(y_test_male, y_pred_male, average='weighted')

print("Performance for Males on the test set:")
print("Balanced Accuracy:", balanced_accuracy_male)
print("Recall:", recall_male)
print("Precision:", precision_male)

# Confusion matrix for males
conf_matrix_male = confusion_matrix(y_test_male, y_pred_male)
plt.rcParams.update({'font.family': 'Times New Roman', 'font.size': 16})

plt.figure()
sns.heatmap(conf_matrix_male, annot=True, fmt='d', cmap='viridis', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Males')
plt.savefig('Plots/confusion_matrix_gradient_boosting_males.pdf')
plt.show()

# Evaluation metrics for females
balanced_accuracy_female = balanced_accuracy_score(y_test_female, y_pred_female)
recall_female = recall_score(y_test_female, y_pred_female, average='weighted')
precision_female = precision_score(y_test_female, y_pred_female, average='weighted')

print("Performance for Females on the test set:")
print("Balanced Accuracy:", balanced_accuracy_female)
print("Recall:", recall_female)
print("Precision:", precision_female)

# Confusion matrix for females
conf_matrix_female = confusion_matrix(y_test_female, y_pred_female)
plt.rcParams.update({'font.family': 'Times New Roman', 'font.size': 16})

plt.figure()
sns.heatmap(conf_matrix_female, annot=True, fmt='d', cmap='viridis', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Females')
plt.savefig('Plots/confusion_matrix_gradient_boosting_females.pdf')
plt.show()

### Exploration of Key Determinants

Consists of:
- Feature Importance Analysis
- Shap Analysis
- Category Importance Analysis

In [2]:
"""
Feature Importance Analysis
"""

with open('best_hyperparameters_gradient_boosting.json', 'r') as file:
    best_hyperparameters_dict = json.load(file)

X_train = pd.read_csv('Datasets/X_train.csv')
y_train = pd.read_csv('Datasets/y_train.csv')
X_train.rename(columns={'distance_to_station_km': 'Dist_station', 'nearest_station_encoded': 'Nearest_station'}, inplace=True)

# Final Gradient Boosting as this is best performing model
best_gradient_boosting = GradientBoostingClassifier(random_state=42, **best_hyperparameters_dict)
best_gradient_boosting.fit(X_train, y_train.values.ravel())

# Calculating Feature Importances
feature_importance_scores = best_gradient_boosting.feature_importances_
feature_importance_dict = dict(zip(X_train.columns, feature_importance_scores))

# Creating a DataFrame to store feature names and importance scores
feature_importance_series = pd.Series(feature_importance_scores)
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance Score': feature_importance_series
})

# Sorting the DataFrame
feature_importance_df = feature_importance_df.sort_values(by='Importance Score', ascending=False)
feature_names = feature_importance_df['Feature'][:20]
importance_scores = feature_importance_df['Importance Score'][:20]

# Define colours
num_colours = 20
viridis_cmap = plt.cm.get_cmap('viridis')
colors = [viridis_cmap(i / num_colours) for i in range(num_colours)]

# Plot feature importance scores
plt.figure(figsize=(14,8))
plt.grid(alpha=0.7, linestyle='dotted', zorder=0)
bars = plt.barh(feature_names, importance_scores, color=colors, zorder=3)
plt.xlabel('Importance Score', fontsize=22, fontname='Times New Roman')
plt.ylabel('Feature', fontsize=22, fontname='Times New Roman')
plt.gca().invert_yaxis()
plt.xticks(fontsize=22, fontname='Times New Roman')
plt.yticks(fontsize=22, fontname='Times New Roman')

# Adding score next to each bar
for bar, score in zip(bars, importance_scores):
    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{score:.3f}', 
             ha='left', va='center', color='black', fontsize=22, fontname='Times New Roman')
# Enlarging the x-axis
plt.xlim(right=max(importance_scores) * 1.1)
plt.tight_layout()
plt.savefig('Plots/feature_importance.pdf')
plt.show()

''

In [ ]:
"""
SHAP Analysis

Adapted from: https://towardsdatascience.com/shap-for-binary-and-multiclass-target-variables-ff2f43de0cf4
"""

with open('best_hyperparameters_gradient_boosting.json', 'r') as file:
    best_hyperparameters_dict = json.load(file)

X_train = pd.read_csv('Datasets/X_train.csv')
y_train = pd.read_csv('Datasets/y_train.csv')
X_test = pd.read_csv('Datasets/X_test.csv')
y_test = pd.read_csv('Datasets/y_test.csv')

# Final Gradient Boosting as this is best performing model
best_gradient_boosting = GradientBoostingClassifier(random_state=42, **best_hyperparameters_dict)
best_gradient_boosting.fit(X_train, y_train.values.ravel())

# Creating a SHAP values
explainer = shap.TreeExplainer(best_gradient_boosting)
shap_values = explainer.shap_values(X_test)

# Storing SHAP values
with open('shap_values.pkl', 'wb') as f:
    pickle.dump(shap_values, f)
with open('shap_values.pkl', 'rb') as f:
    loaded_shap_values = pickle.load(f)

# Converting the NumPy array to an Explanation object
shap_values_explanation = shap.Explanation(values=loaded_shap_values, feature_names=feature_names)

# Getting column names from the training data
feature_names = X_train.columns

# Setting font properties
plt.rcParams.update({'font.family': 'Times New Roman', 
                     'font.size': 24})

# Plot SHAP values
shap.plots.bar(shap_values_explanation[0], max_display=20, show=False)

plt.xlabel("SHAP Value")
plt.ylabel("Feature")
plt.tight_layout()
plt.savefig('Plots/barplot_shap.pdf')

In [ ]:
"""
Category Importance Analysis
"""

X_train = pd.read_csv('Datasets/X_train.csv')
y_train = pd.read_csv('Datasets/y_train.csv')

with open('Datasets/feature_importance_scores.json', 'r') as file:
    feature_importance_scores = json.load(file)

# Category Mappings
category_mappings = {
    'Trip Information': ['BerWrk', 'RdWrkA', 'RdWrkB', 'BerOnd', 'RdOndA', 'RdOndB', 
                         'BerSup', 'RdSupA', 'RdSupB', 'BerZiek', 'RdZiekA', 'RdZiekB', 'BerArts', 
                         'RdArtsA', 'RdArtsB', 'BerStat', 'RdStatA', 'RdStatB', 'BerHalte', 
                         'RdHalteA', 'RdHalteB', 'BerFam', 'RdFamA', 'RdFamB', 'BerSport', 
                         'RdSportA', 'RdSportB', 'Weggeweest', 'RedenNW', 'RedenNWZ', 'RedenNWB', 
                         'RedenNWW', 'AantVpl', 'AantOVVpl', 'AantSVpl',
                         'ReisduurOP', 'AfstandOP', 'AfstandSOP', 'Doel', 'MotiefV', 
                         'KMotiefV', 'AardWerk', 'VertLoc', 'AfstV', 'KAfstV', 'AfstR', 
                         'KAfstR', 'RTSamen', 'AantRit', 'Toer'],
    'Regional Information': ['WoPC', 'WoGem', 'Sted', 'GemGr', 'Prov', 'VertPC', 'VertGem', 
                             'VertProv', 'AankPC', 'AankGem', 'AankProv', 'RVertStat', 'RAankStat', 
                             'Longitude', 'Latitude', 'distance_to_station_km', 'station_type_encoded',
                             'nearest_station_encoded'],
    'Financial Information': ['BetWerk', 'OnbBez', 'HHBestInkG', 'HHGestInkG', 'HHLaagInk', 
                               'HHSocInk', 'HHWelvG', 'OVStKaart', 'WrkVerg', 'VergVast', 'VergKm', 
                               'VergBrSt', 'VergOV', 'VergAans', 'VergVoer', 'VergBudg', 'VergPark', 
                               'VergStal', 'VergAnd'],
    'Demographic Information': ['HHPers', 'HHSam', 'HHPlOP', 'HHLft1', 'HHLft2', 'HHLft3', 'HHLft4', 
                                 'Geslacht', 'Leeftijd', 'KLeeft', 'Herkomst', 'MaatsPart', 'Opleiding', 
                                 'HHRijbewijsAu', 'HHRijbewijsMo', 'HHRijbewijsBr', 'OPRijbewijsAu', 
                                 'OPRijbewijsMo', 'OPRijbewijsBr', 'HHAuto', 'HHAutoL', 'OPAuto', 
                                 'BrandstofPa1', 'XBrandstofPaL', 'XBrandstofPa1', 'BrandstofEPaL', 'BrandstofEPa1', 'BrandstofPaL', 
                                 'HHMotor', 'OPMotor', 'HHBrom', 'OPBrom', 'HHSnor', 'OPSnor', 
                                 'HHEFiets', 'HHBezitVm', 'OPBezitVm', 'FqLopen', 'FqNEFiets', 
                                 'FqEFiets', 'FqBTM', 'FqTrein', 'FqAutoB', 'FqAutoP', 'FqMotor', 
                                 'FqBrSnor', 'Kind6'],
    'Temporal Dynamics': ['Maand', 'Week', 'Dag', 'Weekdag', 'Feestdag', 'VertUur', 'VertMin',
                          'KVertTijd', 'AankUur', 'AankMin', 'Reisduur', 'KReisduur', 'RVertUur', 
                          'RVertMin', 'RAankUur', 'RAankMin', 'RReisduur']
}

# Check if all features are mapped
all_features = set(X_train.columns)
mapped_features = set(feature for features in category_mappings.values() for feature in features)
unmapped_features = all_features - mapped_features
if unmapped_features:
    print("Error: The following variables are not mapped to any category:", ', '.join(unmapped_features))


# Check if all mapped features are present in the dataset
missing_features = mapped_features - all_features
if missing_features:
    print("Error: The following mapped variables are not present in the dataset:", ', '.join(missing_features))

# Category Importance score
category_importance_scores = {}

for category, features in category_mappings.items():
    category_importance_scores[category] = sum([feature_importance_scores[feature] for feature in features])

    sorted_category_importance_scores = dict(sorted(category_importance_scores.items(), key=lambda item: item[1], reverse=True))

# Print category importance scores
print("Category Importance Scores:")
for category, importance_score in sorted_category_importance_scores.items():
    print(f"{category}: {importance_score:.4f}")

# Identifying the most influential feature for each category
most_influential_features = {}
for category, features in category_mappings.items():
    feature_scores = {feature: feature_importance_scores[feature] for feature in features}
    most_influential_features[category] = max(feature_scores, key=feature_scores.get)

# Print the most influential feature for each category
print("\nMost Influential Features from Each Category:")
for category, feature in most_influential_features.items():
    print(f"{category}: {feature} ({feature_importance_scores[feature]:.4f})")

# Category names and their importance scores
category_names = list(sorted_category_importance_scores.keys())
category_scores = list(sorted_category_importance_scores.values())


# Category importances plot
colors = plt.cm.viridis(np.linspace(0, 1, len(category_names)))

plt.figure(figsize=(14, 8))
plt.grid(alpha=0.7, linestyle='dotted', zorder=0)
bars = plt.barh(category_names, category_scores, color=colors, zorder=3)
plt.xlabel('Category Importance Score', fontsize=20, fontname='Times New Roman')
plt.ylabel('Category', fontsize=20, fontname='Times New Roman')
plt.gca().invert_yaxis()

plt.xticks(fontsize=20, fontname='Times New Roman')
plt.yticks(fontsize=20, fontname='Times New Roman')

# Add score to each bar
for bar, score in zip(bars, category_scores):
    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{score:.3f}', 
             ha='left', va='center', color='black', fontsize=20, fontname='Times New Roman')
# Enlarging x-axis for readability
plt.xlim(right=max(category_scores) * 1.1)

plt.tight_layout()
plt.savefig('Plots/category_importance.pdf')
plt.show()